In [68]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml

In [69]:
OUTPUT_FOLDER = '../data/test/'
EPSG = CRS.WGS84
IMAGE_SIZE = 14

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key']

In [70]:
def half_hectare_accuracy(true, pred):
    eps = 1e-9
    true_x = np.split(true, indices_or_sections = 2, axis = 0)
    true_y = [np.split(x, 2, 1) for x in true_x]
    true_xy = [item for sublist in true_y for item in sublist]
    true = [np.sum(x) for x in true_xy]
    
    pred_x = np.split(pred, indices_or_sections = 2, axis = 0)
    pred_y = [np.split(x, 2, 1) for x in pred_x]
    pred_xy = [item for sublist in pred_y for item in sublist]
    pred = [np.sum(x) for x in pred_xy]
    
    recall = [min(x / y, 1) for x, y in zip(pred, true) if y > 0]
    precision = [(x - y) / x for x, y in zip(pred, true)]
    for i, val in enumerate(precision):
        if val < 0:
            precision[i] = 1
    return np.mean(recall), np.mean(precision)    
    
        
half_hectare_accuracy(np.ones((14, 14)), np.zeros((14, 14)))

/Users/john.brandt/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


(0.0, 1.0)

In [71]:
def thirty_meter(true, pred):
    indices_x = np.random.randint(0, 10, 3)
    indices_y = np.random.randint(0, 10, 3)
    indexes = ([([a, a + 3], [b, b + 3]) for a,b in zip(indices_x, indices_y)])
    subs_true = []
    subs_pred = []
    for i in indexes:
        true_i = true[i[0][0]:i[0][1], i[1][0]:i[1][1]]
        pred_i = pred[i[0][0]:i[0][1], i[1][0]:i[1][1]]
        subs_true.append(true_i)
        subs_pred.append(pred_i)
    pred = [np.sum(x) for x in subs_pred]
    true = [np.sum(x) for x in subs_true]
    recall = [min(x / y, 1) for x, y in zip(pred, true) if y > 0]
    precision = [(x - y) / x for x, y in zip(pred, true)]
    for i, val in enumerate(precision):
        if val < 0:
            precision[i] = 1
    return np.mean(recall), np.mean(precision)   

thirty_meter(np.ones((14, 14)), np.zeros((14, 14)))

/Users/john.brandt/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


(0.0, 1.0)

In [72]:
sample_raster = np.zeros((14, 14))
indices_x = [x for x in range(0, 15, 7)]
indices_y = [x for x in range(0, 15, 7)]

x_len = 14 // 7
y_len = 14 // 7
ids_grid = range(0, x_len*y_len)
print(len(ids_grid))

sample_raster = np.split(sample_raster, indices_or_sections = indices_x)

y = [np.split(i, indices_or_sections = indices_y, axis = 1) for i in sample_raster]
flat_list = [item for sublist in y for item in sublist]

4


In [73]:
subs = [x for x in flat_list if x.shape[0] == 14]
subs = [x for x in subs if x.shape[1] == 14]

In [116]:
# setup function to reproject coordinates
def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])

def bounding_box(point, x_offset_max = 140, y_offset_max = 140):
    # LONG, LAT FOR SOME REASON
    tl = point
    
    if 48 <= tl[0] <= 54:
        epsg = 32639 if tl[1] > 0 else 32739
    if 42 <= tl[0] <= 48:
        epsg = 32638 if tl[1] > 0 else 32738
    if 36 <= tl[0] <= 42:
        epsg = 32637 if tl[1] > 0 else 32737
    if 30 <= tl[0] <= 36:
        epsg = 32636 if tl[1] > 0 else 32736
    if 24 <= tl[0] <= 30:
        epsg = 32635 if tl[1] > 0 else 32735
    if 18 <= tl[0] <= 24:
        epsg = 32634 if tl[1] > 0 else 32734

    tl = convertCoords(tl, 4326, epsg)
    
    br = (tl[0] + x_offset_max, tl[1] + y_offset_max)
    tl = ((tl[0] + (x_offset_max - 140)), (tl[1] + (y_offset_max - 140)))
    br = convertCoords(br, epsg, 4326)
    tl = convertCoords(tl, epsg, 4326)
    
    min_x = tl[0]
    max_x = br[0]
    
    min_y = tl[1]
    max_y = br[1]
    # (min_x, min_y), (max_x, max_y)
    # (bl, tr)
    return [(min_x, min_y), (max_x, max_y)]

In [188]:
# Test 1 == 13.727559, 38.316348
# Test 1.1 == , 38.246539, 13.934339
# Test 1.2 village with trees 39.532886, 12.870031
# Test 1.3 crop, trees 13.685493, 38.175899
# Test 1.4 trees on grass 13.584549, 38.179959
# Trees on farm 13.577898, 38.174085
# Segmented road 14.165738, 38.156894

# NEW 13.898518, 39.063205
# Uganda trees on farms 2.123535, 33.679550
# Rwanda marsh -2.059023, 29.981993

# 8 x 9 grid 14.190815, 38.151069

# 16 x 18 grid 14.245881, 38.173931
# 16 x 18 grid 2 14.318609, 38.141159
# 16 x 9 grid 14.346171, 38.167957
# 32 x 20 grid 14.311645, 38.121082
coords = (14.180693, 38.141953)
coords = (coords[1], coords[0])
print(coords)
# Test 2 -- village 38.33024, 13.715228
# Test 3 -- farm 13.487358, 39.095307
#initial_point = bounding_box((39.095307, 13.487358))
initial_point = bounding_box(coords)
print(initial_point)

(38.141953, 14.180693)
[(38.141952999999994, 14.180692999999996), (38.143245528203266, 14.181963358704197)]


In [189]:
import itertools
TEST_X = 24
TEST_Y = 15
test = []

offsets_x = [x for x in range(140, 140*(TEST_X + 1), 140)]
offsets_y = [x for x in range(140, 140*(TEST_Y + 1), 140)]
perms = [(y, x) for x, y in itertools.product(offsets_y, offsets_x)]

for i in perms:
    bbx = bounding_box(initial_point[0], y_offset_max = i[1], x_offset_max = i[0])
    test.append(bbx)

In [190]:
min_x = []
max_x = []
min_y = []
max_y = []
for x in test:
    min_x.append(min((x[0][0], x[1][0])))
    max_x.append(max(x[0][0], x[1][0]))
    min_y.append(min(x[0][1], x[1][1]))
    max_y.append(max(x[0][1], x[1][1]))
    
print("West {} South {} East {} North {}".format(min(min_x), min(min_y), max(max_x), max(max_y)))

West 38.14188631704652 South 14.180692999999994 East 38.173083285129145 North 14.199788339267883


In [191]:
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

# cloud bands return [B01,B02,B04,B05,B08,B8A,B09,B10,B11,B12]
# ndvi bands [B02,B03,B04,B05,B06,B07, B08, B8A,B11,B12]
# updated ndvi = [1, 2, 3, 4, 5, 6, 7, 8, 8a, 9, 10, 11, 12]
# cloud bands [y, y, n, y, y, n, n, y, y, y, y, y, y]
# ndvi bands [n, y, y, y, y, y, y, y, n, n, y, y]

def identify_clouds(bbox, epsg = EPSG, time = ('2017-01-01', '2017-12-31')):
    try:
        box = BBox(bbox, crs = epsg)
        cloud_request = WmsRequest(
            layer='CLOUD_DETECTION',
            bbox=box,
            time=time,
            width=IMAGE_SIZE,
            height=IMAGE_SIZE,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.8,
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
            time_difference=datetime.timedelta(hours=24),
        )
        
        cloud_img = cloud_request.get_data()
        cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
        means = np.mean(cloud_probs, (1, 2))
        cloud_steps = [i for i, val in enumerate(means) if val > 0.25]
        return cloud_steps, means
    except Exception as e:
        logging.fatal(e, exc_info=True)
    
        
    
def download_tiles(bbox, epsg = EPSG, time = ('2017-01-01', '2017-12-31')):
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WmsRequest(
                layer='ALL_BANDS_NDVI',
                bbox=box,
                time=time,
                width=IMAGE_SIZE,
                height=IMAGE_SIZE,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.8,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=24),
            )
        img_bands = image_request.get_data()
        return img_bands, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)
    

def calculate_and_save_best_images(cloud_steps, img_bands, image_request, means):
    begining_length = len(img_bands)
    clean_steps = np.array([x for x in range(len(img_bands)) if x not in cloud_steps])
    keep_steps = []
    month_steps = []
    month_hash = []
    for date in image_request.get_dates():
         month_steps.append(date.month)
            
    # Identify two images per month with the least cloud cover
    best_two_per_month = []
    for i in range(1, 13):
        month_i = []
        month_i_clouds = []
        for position, item in enumerate(month_steps):
            if item == i:
                month_i.append(position)
        clouds = [val for x, val in enumerate(means) if x in month_i]
        if len(clouds) > 2:
            clouds = sorted(clouds)[:2]
            ids = [x for x, val in enumerate(means) if val in clouds]
        else:
            ids = month_i_clouds
        for x in ids:
            best_two_per_month.append(x)
                    
        # Append the best two per month
    for i in best_two_per_month:
        counts = Counter([x for x in month_hash])
        current_month_count = counts.get(month_steps[i])
        if current_month_count == None:
            current_month_count = 0
        if i not in cloud_steps and current_month_count <= 1: 
            keep_steps.append(img_bands[i])
            month_hash.append(month_steps[i])
        if i in cloud_steps and min(clean_steps) < i < max(clean_steps):
            if current_month_count <= 1:
                nearest_lower = clean_steps[clean_steps > i].min()
                nearest_upper = clean_steps[clean_steps < i].max()
                img_bands[i] = (img_bands[nearest_lower] + img_bands[nearest_upper])/2
                month_hash.append(month_steps[i])
                keep_steps.append(img_bands[i])
    npify = np.stack(keep_steps)
    print("{}; removed {} steps of {} initial".format(npify.shape, len(cloud_steps), begining_length))
    return(npify)

In [192]:
errors = []
print("There are {} files to download".format(len(test)))
for i in range(0, len(test)):
    print(i)
    try:
        # Initiate hash tables
        cloud, means = identify_clouds(test[i])
        img, image_request = download_tiles(test[i])
        tiles = calculate_and_save_best_images(cloud, img, image_request, means)
        np.save(OUTPUT_FOLDER + str(i), tiles)

    except Exception as e:
        logging.fatal(e, exc_info=True)
        errors.append(img)
        #continue

There are 360 files to download
0
(22, 14, 14, 10); removed 12 steps of 49 initial
1
(22, 14, 14, 10); removed 12 steps of 49 initial
2
(22, 14, 14, 10); removed 13 steps of 49 initial
3
(22, 14, 14, 10); removed 13 steps of 49 initial
4
(22, 14, 14, 10); removed 11 steps of 49 initial
5
(22, 14, 14, 10); removed 13 steps of 49 initial
6
(22, 14, 14, 10); removed 11 steps of 49 initial
7
(22, 14, 14, 10); removed 11 steps of 49 initial
8
(22, 14, 14, 10); removed 11 steps of 49 initial
9
(22, 14, 14, 10); removed 12 steps of 49 initial
10
(22, 14, 14, 10); removed 11 steps of 49 initial
11
(22, 14, 14, 10); removed 11 steps of 49 initial
12
(22, 14, 14, 10); removed 9 steps of 49 initial
13
(22, 14, 14, 10); removed 11 steps of 49 initial
14
(22, 14, 14, 10); removed 12 steps of 49 initial
15
(22, 14, 14, 10); removed 11 steps of 49 initial
16
(22, 14, 14, 10); removed 9 steps of 49 initial
17
(22, 14, 14, 10); removed 7 steps of 49 initial
18
(22, 14, 14, 10); removed 8 steps of 48 in

(22, 14, 14, 10); removed 12 steps of 49 initial
159
(22, 14, 14, 10); removed 9 steps of 49 initial
160
(22, 14, 14, 10); removed 8 steps of 49 initial
161
(22, 14, 14, 10); removed 9 steps of 49 initial
162
(22, 14, 14, 10); removed 10 steps of 48 initial
163
(22, 14, 14, 10); removed 10 steps of 48 initial
164
(22, 14, 14, 10); removed 10 steps of 48 initial
165
(22, 14, 14, 10); removed 9 steps of 48 initial
166
(22, 14, 14, 10); removed 9 steps of 48 initial
167
(22, 14, 14, 10); removed 7 steps of 48 initial
168
(22, 14, 14, 10); removed 12 steps of 49 initial
169
(22, 14, 14, 10); removed 11 steps of 49 initial
170
(22, 14, 14, 10); removed 9 steps of 49 initial
171
(22, 14, 14, 10); removed 10 steps of 49 initial
172
(22, 14, 14, 10); removed 10 steps of 49 initial
173
(22, 14, 14, 10); removed 9 steps of 49 initial
174
(22, 14, 14, 10); removed 9 steps of 49 initial
175
(22, 14, 14, 10); removed 10 steps of 49 initial
176
(22, 14, 14, 10); removed 11 steps of 49 initial
177
(2

(22, 14, 14, 10); removed 11 steps of 49 initial
315
(22, 14, 14, 10); removed 11 steps of 49 initial
316
(22, 14, 14, 10); removed 12 steps of 49 initial
317
(22, 14, 14, 10); removed 11 steps of 49 initial
318
(22, 14, 14, 10); removed 12 steps of 49 initial
319
(22, 14, 14, 10); removed 14 steps of 49 initial
320
(22, 14, 14, 10); removed 13 steps of 49 initial
321
(22, 14, 14, 10); removed 14 steps of 49 initial
322
(22, 14, 14, 10); removed 12 steps of 49 initial
323
(22, 14, 14, 10); removed 12 steps of 49 initial
324
(22, 14, 14, 10); removed 13 steps of 49 initial
325
(22, 14, 14, 10); removed 13 steps of 49 initial
326
(22, 14, 14, 10); removed 13 steps of 49 initial
327
(22, 14, 14, 10); removed 11 steps of 49 initial
328
(22, 14, 14, 10); removed 9 steps of 49 initial
329
(22, 14, 14, 10); removed 11 steps of 49 initial
330
(22, 14, 14, 10); removed 11 steps of 48 initial
331
(22, 14, 14, 10); removed 11 steps of 48 initial
332
(22, 14, 14, 10); removed 12 steps of 48 initia